# Test spectral and histogram loss for 3D images
Jan 6, 2021

In [35]:
import os
import random
import logging
import sys

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
import torch.fft

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import argparse
import time
from datetime import datetime
import glob
import pickle
import yaml
import collections

OSError: libcurand.so.10: cannot open shared object file: No such file or directory

In [2]:
####################
### Pytorch code ###
####################

def f_get_rad(img):
    ''' Get the radial tensor for use in f_torch_get_azimuthalAverage '''
    
    height,width,depth=img.shape[-3:]
    # Create a grid of points with x and y coordinates
    z,y,x = np.indices([height,width,depth])
    
    center=[]
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r= torch.tensor(np.sqrt((x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2))
        
    # Get sorted radii
    ind = torch.argsort(torch.reshape(r, (-1,)))

    return r.detach(),ind.detach()

def f_torch_get_azimuthalAverage(image,r,ind):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    
#     height,width,depth=img.shape[-3:]
#     # Create a grid of points with x and y coordinates
#     z,y,x = np.indices([height,width,depth])
    
#     center=[]
#     if not center:
#         center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0])

#     # Get the radial coordinate for every grid point. Array has the shape of image
#     r= torch.tensor(np.sqrt((x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2))
        
#     # Get sorted radii
#     ind = torch.argsort(torch.reshape(r, (-1,)))

    r_sorted = torch.gather(torch.reshape(r, ( -1,)),0, ind)
    i_sorted = torch.gather(torch.reshape(image, ( -1,)),0, ind)
    
    # Get the integer part of the radii (bin size = 1)
    r_int=r_sorted.to(torch.int32)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = torch.reshape(torch.where(deltar)[0], (-1,))    # location of changes in radius
    nr = (rind[1:] - rind[:-1]).type(torch.float)       # number of radius bin

    # Cumulative sum to figure out sums for each radius bin
    
    csum = torch.cumsum(i_sorted, axis=-1)
    tbin = torch.gather(csum, 0, rind[1:]) - torch.gather(csum, 0, rind[:-1])
    radial_prof = tbin / nr

    return radial_prof

def f_torch_fftshift(real, imag):
    print(real.shape,imag.shape,real.size(),real.size(0))
    for dim in range(0, len(real.size())):
        real = torch.roll(real, dims=dim, shifts=real.size(dim)//2)
        imag = torch.roll(imag, dims=dim, shifts=imag.size(dim)//2)
    return real, imag

def f_torch_compute_spectrum(arr,r,ind):
    
    GLOBAL_MEAN=1.0
    arr=(arr-GLOBAL_MEAN)/(GLOBAL_MEAN)
    
#     y1=torch.rfft(arr,signal_ndim=3,onesided=False) ## 3D FFT
#     real,imag=f_torch_fftshift(y1[:,:,:,0],y1[:,:,:,1])    ## last index is real/imag part
    
    y1=torch.fft.fftn(arr,dim=(-3,-2,-1))
    real,imag=f_torch_fftshift(y1.real,y1.imag)
    y2=real**2+imag**2     ## Absolute value of each complex number
    z1=f_torch_get_azimuthalAverage(y2,r,ind)     ## Compute radial profile
    return z1


def f_torch_compute_batch_spectrum(arr,r,ind):
    
    batch_pk=torch.stack([f_torch_compute_spectrum(i,r,ind) for i in arr])
    
    return batch_pk

def f_torch_image_spectrum(x,num_channels,r,ind):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    mean=[[] for i in range(num_channels)]    
    var=[[] for i in range(num_channels)]    
    
    print(x.shape)
    for i in range(num_channels):
        arr=x[:,i,:,:,:]
        print(arr.shape)
        batch_pk=f_torch_compute_batch_spectrum(arr,r,ind)
        print(batch_pk.shape)
        mean[i]=torch.mean(batch_pk,axis=0)
        var[i]=torch.var(batch_pk,axis=0)
    
    mean=torch.stack(mean)
    var=torch.stack(var)
        
    return mean,var

def f_compute_hist(data,bins):
    
    try: 
        hist_data=torch.histc(data,bins=bins)
        ## A kind of normalization of histograms: divide by total sum
        hist_data=(hist_data*bins)/torch.sum(hist_data)
    except Exception as e:
        print(e)
        hist_data=torch.zeros(bins)

    return hist_data

### Losses 
def loss_spectrum(spec_mean,spec_mean_ref,spec_std,spec_std_ref,image_size,lambda1):
    ''' Loss function for the spectrum : mean + variance 
    Log(sum( batch value - expect value) ^ 2 )) '''
    
    idx=int(image_size/2) ### For the spectrum, use only N/2 indices for loss calc.
    ### Warning: the first index is the channel number.For multiple channels, you are averaging over them, which is fine.
        
    spec_mean=torch.log(torch.mean(torch.pow(spec_mean[:,:idx]-spec_mean_ref[:,:idx],2)))
    spec_sdev=torch.log(torch.mean(torch.pow(spec_std[:,:idx]-spec_std_ref[:,:idx],2)))
    
    lambda1=lambda1;
    lambda2=lambda1;
    ans=lambda1*spec_mean+lambda2*spec_sdev
    
    if torch.isnan(spec_sdev).any():    print("spec loss with nan",ans)
    
    return ans
    
def loss_hist(hist_sample,hist_ref):
    
    lambda1=1.0
    return lambda1*torch.log(torch.mean(torch.pow(hist_sample-hist_ref,2)))


In [16]:
## numpy code
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    z, y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    center=[]
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0, (z.max()-z.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2+ + (z - center[2])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile[1:-1]

def f_compute_spectrum(arr):
    
    GLOBAL_MEAN=1.0
    arr=((arr - GLOBAL_MEAN)/GLOBAL_MEAN)
    y1=np.fft.fftn(arr)
    y1=np.fft.fftshift(y1)
#     print(y1.shape)
    y2=abs(y1)**2
    z1=f_radial_profile(y2)
    return(z1)
   
def f_compute_batch_spectrum(arr):
    batch_pk=np.array([f_compute_spectrum(i) for i in arr])
    return batch_pk

def f_image_spectrum(x,num_channels):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    mean=[[] for i in range(num_channels)]    
    var=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:,:]
#         print(i,arr.shape)
        batch_pk=f_compute_batch_spectrum(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        var[i]=np.var(batch_pk,axis=0)
    mean=np.array(mean)
    var=np.array(var)
    return mean,var

In [17]:
## Read input
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/full_with_smoothing_1.npy'
img=np.load(ip_fname,mmap_mode='r')[:20]
# img=np.expand_dims(img,axis=1).astype(float)
t_img=torch.from_numpy(img)
print(t_img.shape)

torch.Size([20, 1, 64, 64, 64])


In [18]:
np_mean,np_var=f_image_spectrum(img,1)
r,ind=f_get_rad(t_img)
print(r.shape,ind.shape)
trch_mean,trch_var=f_torch_image_spectrum(t_img,1,r,ind)

torch.Size([64, 64, 64]) torch.Size([262144])
torch.Size([20, 1, 64, 64, 64])
torch.Size([20, 64, 64, 64])
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) torch.Size([64, 64, 64]) 64
torch.Size([64, 64, 64]) torch.Size

In [8]:
# def f_torch_fftshift(real, imag):
#     for dim in range(0, len(real.size())):
#         real = torch.roll(real, dims=dim, shifts=real.size(dim)//2)
#         imag = torch.roll(imag, dims=dim, shifts=imag.size(dim)//2)
#     return real, imag
# trch_mean,trch_var=f_torch_image_spectrum(t_img[:1],1,r,ind)

In [24]:
## Comparison of numpy and pytorch results
print(np.allclose(np_mean,trch_mean.numpy(),rtol=1e-2,atol=1e-8))
print(np.allclose(np_var,trch_var.numpy(),rtol=1e-1,atol=1e-8))

for i in range(np_var.shape[1]):
    b=trch_var.numpy()
    a=np_var[0,i];b=trch_var.numpy()[0,i]
    if (np.abs(a-b)/(a) >0.1 ):     print(a,b)

True
True


In [60]:
np_mean,trch_mean,np_var,trch_var

(array([[5.10719086e+08, 2.91767463e+08, 1.77334734e+08, 1.28270119e+08,
         9.94572520e+07, 7.87818523e+07, 6.28514297e+07, 5.18244073e+07,
         4.17857302e+07, 3.48175773e+07, 2.94484534e+07, 2.43341285e+07,
         2.02252346e+07, 1.65113071e+07, 1.37114477e+07, 1.12951316e+07,
         9.39835009e+06, 7.71050217e+06, 6.39005970e+06, 5.23646511e+06,
         4.31519587e+06, 3.58246827e+06, 3.00170705e+06, 2.49097110e+06,
         2.08720952e+06, 1.77080074e+06, 1.50412270e+06, 1.28634519e+06,
         1.10953798e+06, 9.75612599e+05, 8.64029137e+05, 7.31952770e+05,
         5.98999974e+05, 4.86712463e+05, 4.04388042e+05, 3.31951601e+05,
         2.79500569e+05, 2.35696127e+05, 2.00900660e+05, 1.71240670e+05,
         1.52199265e+05, 1.28968247e+05, 1.13715134e+05, 9.46605564e+04,
         7.69839578e+04, 6.30120982e+04, 5.26664982e+04, 4.54706596e+04,
         4.13918847e+04, 3.67561613e+04, 3.38007635e+04, 3.34661198e+04,
         3.08665671e+04]]),
 tensor([[5.1072e+08, 2